In [ ]:
!pip install --upgrade numpy --target ./python
!pip install --upgrade numexpr --target ./python

In [ ]:
import sys
sys.path.append(r"./python")

import os
import json
import traceback
import urllib.parse
import boto3
from datetime import datetime
import time
from python.smart_search_dataload import SmartSearchDataload

port = 443
bulk_size = 10000000

sm_client = boto3.client('secretsmanager')
master_user = sm_client.get_secret_value(SecretId='opensearch-host-url')['SecretString']
data= json.loads(master_user)
es_host_name = data.get('host')
host = es_host_name+'/' if es_host_name[-1] != '/' else es_host_name# cluster endpoint, for example: my-test-domain.us-east-1.es.amazonaws.com/
host = host[8:-1]
region = boto3.Session().region_name # e.g. cn-north-1
print('host:',host)
print('region:',region)

# retrieve secret manager value by key using boto3                                             
sm_client = boto3.client('secretsmanager')
master_user = sm_client.get_secret_value(SecretId='opensearch-master-user')['SecretString']
data= json.loads(master_user)
username = data.get('username')
password = data.get('password')

In [ ]:
#导入文件在AOS的index名称
index =  "data_index_test_2"   

#导入文件的主要语种，目前支持 chinese 和 english 共2种
language = "chinese"

#部署在SageMaker的向量模型的endpoint名称，当值为：bedrock-titan-embed 时，使用bedrock的amazon.titan-embed-text-v1模型, 当值为bedrock-cohere-embed 时，
#使用cohere.embed-multilingual-v3模型
embedding_endpoint_name = "huggingface-inference-eb"

#需要导入文件的存储路径
local_path = "../docs/"

#目前主要在英语文档拆分时用到，拆分后文档的最大token数量
chunk_size = 1500

#目前主要在英语文档拆分时用到，拆分后相邻文档的头尾最大重合token数量
chunk_overlap = 10 

#目前主要在拆分csv文档时用到，一行的内容如果token数超过该参数，就会对一行内容进行拆分
sep_word_len = 2000  

#目前主要在拆分QA格式的csv文档时用到，如果指定了title_name，在拆分时会确保单独对title列的文本进行向量化，在split_to_sentence_paragraph=True时使用
qa_title_name = ''     

#文档是否拆分为sentence、paragraph的格式，使用sentence文本进行向量化，使用多条sentence组合为paragraph，给到大模型推理
split_to_sentence_paragraph = True    

#使用多少条sentence组装为paragraph
paragraph_include_sentence_num = 3  

#需要向量化文本的最大字数，使用SageMaker部署的向量模型时使用
text_max_length = 350   

#PDF格式的文件使用，设置为true时会先将PDF文件转换为HTML文件进行逻辑段落的拆分,在split_to_sentence_paragraph=True时使用
pdf_to_html = False    

#写入AOS的文本字段名称，langchain默认为text
text_field = 'paragraph' 

#写入AOS的向量字段名称，langchain默认为vector_field
vector_field = 'sentence_vector'

In [ ]:
sys.path.append(r"./python")

search_qa = SmartSearchDataload()
search_qa.init_cfg(index,
                 username,
                 password,
                 host,
                 port,
                 region,
                 embedding_endpoint_name,
                 language=language
                 )
files = os.listdir(local_path)
i = 0
for file in files:
    local_file = local_path+file
    now1 = datetime.now()#begin time
    loaded_files = search_qa.init_knowledge_vector(local_file,
                                                   chunk_size,
                                                   chunk_overlap,
                                                   sep_word_len,
                                                   qa_title_name,
                                                   split_to_sentence_paragraph,
                                                   paragraph_include_sentence_num,
                                                   text_max_length,
                                                   pdf_to_html,
                                                   text_field,
                                                   vector_field)
    now2 = datetime.now()#endtime
    i += 1
    print(i," Complete the import of the following documents:", str(loaded_files)," File import takes time:",now2-now1 )

